# Group Project ProposalHeart Disease

### Introduction



Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal
Clearly state the question you will try to answer with your project
Identify and describe the dataset that will be used to answer the ques

tion
Preliminary exploratory data analysis:
Demonstrate that the dataset can be read from the web into R 
Clean and wrangle your data into a tidy format
Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 
Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your 


analysis.
Methods:
Explain how you will conduct either your data analysis and which variables/columns you will use. Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?
Describe at least one way that you will visualize

 the results
Expected outcomes and significance:
What do you expect to find?
What impact could such findings have?
What future questions could this lead to?

In [36]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

Introduction:

Heart disease refers to a range of conditions that affect the heart and the term is often used interchangeably with "cardiovascular disease," which generally refers to conditions that involve narrowed or blocked blood vessels that can lead to a heart attack. Heart disease is also a leading cause of death globally, one in five deaths in the U.S. is caused by heart disease and heart disease cost the U.S. 239.9 billion dollars each year. 



In [62]:
# cleaveland_data <- read_tsv("data/cleve.mod", skip = 20)

url <- "https://raw.githubusercontent.com/ANGUO17/dsci-100-2023w2-group-06/main/heart%2Bdisease/reprocessed.hungarian.data"

data <- read.table(url) 

    
head(data)




,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<int>,<int>
1,40,1,2,140,289,0,0,172,0,0.0,-9,-9,-9,0
2,49,0,3,160,180,0,0,156,0,1.0,2,-9,-9,1
3,37,1,2,130,283,0,1,98,0,0.0,-9,-9,-9,0
4,48,0,4,138,214,0,0,108,1,1.5,2,-9,-9,3
5,54,1,3,150,-9,0,0,122,0,0.0,-9,-9,-9,0
6,39,1,3,120,339,0,0,170,0,0.0,-9,-9,-9,0
